In [7]:
import cv2
import numpy as np
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
from ultralytics import YOLO
import os
import sqlite3
import insightface
from insightface.app import FaceAnalysis

# =====================
# Configuration Settings
# =====================

IMG_SIZE = 224
FACE_THRESHOLD = 0.65
GPU_MEM_LIMIT = 1536  # MB
DATABASE_FILE = "face_database.db"
MIN_CAPTURE_INTERVAL = 5  # seconds

# =====================
# Initialize Models
# =====================
app = FaceAnalysis(name="buffalo_l", providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])

app.prepare(ctx_id=0)
# GPU Configuration
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=GPU_MEM_LIMIT)]
        )
    except RuntimeError as e:
        print(f"GPU Config Error: {e}")

# Face Detection Model
face_detector = cv2.dnn.readNetFromCaffe(
    "deploy.prototxt", 
    "res10_300x300_ssd_iter_140000_fp16.caffemodel"
)

# Image Classification Model
classification_model = tf.keras.applications.ResNet50V2(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax'
)

# =====================
# Database Functions
# =====================
def init_db():
    """Initialize SQLite database"""
    conn = sqlite3.connect(DATABASE_FILE)
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS faces 
                (id INTEGER PRIMARY KEY, name TEXT, embedding BLOB)''')
    conn.commit()
    conn.close()

def save_to_db(name, embedding):
    """Save face embedding to database"""
    conn = sqlite3.connect(DATABASE_FILE)
    c = conn.cursor()
    c.execute("INSERT INTO faces (name, embedding) VALUES (?, ?)",
             (name, embedding.tobytes()))
    conn.commit()
    conn.close()

def load_from_db():
    """Load all face embeddings from database"""
    conn = sqlite3.connect(DATABASE_FILE)
    c = conn.cursor()
    c.execute("SELECT name, embedding FROM faces")
    data = {row[0]: np.frombuffer(row[1], dtype=np.float32) for row in c.fetchall()}
    conn.close()
    return data

# =====================
# Image Processing
# =====================
def process_face(image):
    """Preprocess face image for embedding"""
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    return (image / 127.5) - 1.0

def get_embedding(image):
    """Generate face embedding using InsightFace"""
    faces = app.get(image)
    if faces:
        return faces[0].embedding
    else:
        return None

def classify_image(image):
    """Classify image using ResNet50V2"""
    img = cv2.resize(image, (224, 224))
    img = tf.keras.applications.resnet_v2.preprocess_input(img)
    preds = classification_model.predict(np.expand_dims(img, 0))
    return tf.keras.applications.resnet_v2.decode_predictions(preds, top=3)[0]

def similarity_search(query_img, threshold):
    """Search for similar faces in database"""
    embedding = get_embedding(query_img)
    if embedding is None:
        return []
    
    database = load_from_db()
    
    results = []
    for name, db_emb in database.items():
        similarity = cosine_similarity([embedding], [db_emb])[0][0]
        if similarity >= threshold:
            results.append((name, similarity))
    
    return sorted(results, key=lambda x: x[1], reverse=True)


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/sina/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/sina/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/sina/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/sina/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/sina/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size

In [8]:
def similarity_search_from_image(image_path,thresh):
    image = cv2.imread(image_path)
    blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0))
    face_detector.setInput(blob)
    detections = face_detector.forward()

    faces = []
    for i in range(detections.shape[2]):
        confidence = detections[0,0,i,2]
        if confidence > 0.9:
            box = detections[0,0,i,3:7] * np.array([image.shape[1], image.shape[0]]*2)
            faces.append(box.astype("int"))

    for (x, y, w, h) in faces:
        face_img = image[y:h, x:x+w]
        
    return similarity_search(face_img,thresh)

In [9]:
def webcam_face_identifier():
    init_db()

    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    db = load_from_db()

    register_mode = False
    embeddings = []
    current_name = ""

    
    
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))
        face_detector.setInput(blob)
        detections = face_detector.forward()

        faces = []
        for i in range(detections.shape[2]):
            confidence = detections[0,0,i,2]
            if confidence > 0.9:
                box = detections[0,0,i,3:7] * np.array([frame.shape[1], frame.shape[0]]*2)
                faces.append(box.astype("int"))

        for (x, y, w, h) in faces:
            face_img = frame[y:h, x:x+w]
            
            if register_mode:
                embeddings.append(get_embedding(face_img))
                cv2.putText(frame, f"Captured: {len(embeddings)}/50", (x+5,y-30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
            else:
                emb = get_embedding(face_img)
                best_match = ("Unknown", 0.0)
                for name, db_emb in db.items():
                    similarity = cosine_similarity([emb], [db_emb])[0][0]
                    if similarity > best_match[1]:
                        best_match = (name, similarity)
                
                color = (0,255,0) if best_match[1] > FACE_THRESHOLD else (0,0,255)
                cv2.rectangle(frame, (x,y), (w,h), color, 2)
                cv2.putText(frame, f"{best_match[0]} ({best_match[1]:.0%})", 
                           (x+5,y-30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        db = load_from_db()                
        
        cv2.imshow('Smart Vision System', frame)
        
        key = cv2.waitKey(1)
        if key == ord('q'):
            break
        elif key == ord('s'):
            current_name = input("Enter name: ")
            register_mode = True
        elif key == ord(' ') and register_mode and len(embeddings) >= 50:
            avg_emb = np.mean(embeddings, axis=0)
            save_to_db(current_name, avg_emb)
            db = load_from_db()
            register_mode = False
            embeddings = []

    cap.release()
    cv2.destroyAllWindows()

In [10]:
webcam_face_identifier()

/home/sina/anaconda3/envs/tf/lib/python3.12/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (73,) + inhomogeneous part.

In [6]:
def object_detection_from_webcam():
    
    model_path = 'yolo11m.pt' 
    model = YOLO(model_path)

     
    cap = cv2.VideoCapture(0)  
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        
        results = model.predict(frame,)

        
        for result in results:
            boxes = result.boxes
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0]
                class_id = box.cls[0]
                confidence = box.conf[0]

                if confidence > 0.5:
                    label = model.names[int(class_id)]
                    color = (0, 255, 0)
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
                    cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        
        cv2.imshow('Object Detection', frame)

        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break


    cap.release()
    cv2.destroyAllWindows()

In [7]:
object_detection_from_webcam()


0: 480x640 1 person, 177.1ms
Speed: 5.8ms preprocess, 177.1ms inference, 368.9ms postprocess per image at shape (1, 3, 480, 640)



QObject::moveToThread: Current thread (0x41274140) is not the object's thread (0x336baf50).
Cannot move to target thread (0x41274140)

QObject::moveToThread: Current thread (0x41274140) is not the object's thread (0x336baf50).
Cannot move to target thread (0x41274140)

QObject::moveToThread: Current thread (0x41274140) is not the object's thread (0x336baf50).
Cannot move to target thread (0x41274140)

QObject::moveToThread: Current thread (0x41274140) is not the object's thread (0x336baf50).
Cannot move to target thread (0x41274140)

QObject::moveToThread: Current thread (0x41274140) is not the object's thread (0x336baf50).
Cannot move to target thread (0x41274140)

QObject::moveToThread: Current thread (0x41274140) is not the object's thread (0x336baf50).
Cannot move to target thread (0x41274140)

QObject::moveToThread: Current thread (0x41274140) is not the object's thread (0x336baf50).
Cannot move to target thread (0x41274140)

QObject::moveToThread: Current thread (0x41274140) is n

0: 480x640 1 person, 82.4ms
Speed: 5.1ms preprocess, 82.4ms inference, 7.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 82.8ms
Speed: 5.0ms preprocess, 82.8ms inference, 8.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 82.8ms
Speed: 5.0ms preprocess, 82.8ms inference, 9.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 83.0ms
Speed: 6.4ms preprocess, 83.0ms inference, 5.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 83.1ms
Speed: 6.9ms preprocess, 83.1ms inference, 8.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 83.0ms
Speed: 7.6ms preprocess, 83.0ms inference, 8.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 83.3ms
Speed: 6.0ms preprocess, 83.3ms inference, 7.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 84.0ms
Speed: 6.4ms preprocess, 84.0ms inference, 4.8ms postprocess per image at shape (1, 3, 480

In [8]:

def object_detection_from_image(image_path):

    model_path = "yolo11m.pt"
    
    model = YOLO(model_path)

    
    img = cv2.imread(image_path)

  
    results = model.predict(img,)


    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            class_id = box.cls[0]
            confidence = box.conf[0]

            if confidence > 0.5:
                label = model.names[int(class_id)]
                color = (0, 255, 0)
                cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
                cv2.putText(img, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    cv2.imshow('Object Detection', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    # return img,label

image_path = '/home/sina/Desktop/prj/face_recognition/1234.png'  


object_detection_from_image(image_path)
# cv2.imshow('Object Detection', result_image)

# cv2.waitKey(0)
# cv2.destroyAllWindows()


0: 640x544 1 cat, 190.1ms
Speed: 6.8ms preprocess, 190.1ms inference, 4.2ms postprocess per image at shape (1, 3, 640, 544)


QObject::moveToThread: Current thread (0x41274140) is not the object's thread (0x336baf50).
Cannot move to target thread (0x41274140)

QObject::moveToThread: Current thread (0x41274140) is not the object's thread (0x336baf50).
Cannot move to target thread (0x41274140)

QObject::moveToThread: Current thread (0x41274140) is not the object's thread (0x336baf50).
Cannot move to target thread (0x41274140)

QObject::moveToThread: Current thread (0x41274140) is not the object's thread (0x336baf50).
Cannot move to target thread (0x41274140)

QObject::moveToThread: Current thread (0x41274140) is not the object's thread (0x336baf50).
Cannot move to target thread (0x41274140)

QObject::moveToThread: Current thread (0x41274140) is not the object's thread (0x336baf50).
Cannot move to target thread (0x41274140)

QObject::moveToThread: Current thread (0x41274140) is not the object's thread (0x336baf50).
Cannot move to target thread (0x41274140)

QObject::moveToThread: Current thread (0x41274140) is n

In [10]:
classify_image(cv2.imread("1234.jpg"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step


[('n02123159', 'tiger_cat', 0.5627914),
 ('n02123045', 'tabby', 0.3619578),
 ('n02124075', 'Egyptian_cat', 0.07494959)]

In [11]:
similarity_search_from_image("/home/sina/Desktop/prj/Pasted image.png",0.5)

[('sina', 0.5608107)]

In [12]:
print("Available GPUs:", tf.config.list_physical_devices('GPU'))

Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [14]:
import torch

In [15]:
print("CUDA available:", torch.cuda.is_available())

CUDA available: True


In [2]:
import cv2
import numpy as np
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
from ultralytics import YOLO
import os
import sqlite3
import insightface

# =====================
# Configuration Settings
# =====================

IMG_SIZE = 112  # InsightFace models typically use 112x112 images
FACE_THRESHOLD = 0.65
GPU_MEM_LIMIT = 1536  # MB
DATABASE_FILE = "face_database.db"
CAPTURE_DIR = "detected_faces"
MIN_CAPTURE_INTERVAL = 5  # seconds

# =====================
# Initialize Models
# =====================

# GPU Configuration
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=GPU_MEM_LIMIT)]
        )
    except RuntimeError as e:
        print(f"GPU Config Error: {e}")

# Face Detection Model
face_detector = cv2.dnn.readNetFromCaffe(
    "deploy.prototxt",
    "res10_300x300_ssd_iter_140000_fp16.caffemodel"
)

# Face Embedding Model
face_model = insightface.app.FaceAnalysis()
face_model.prepare(ctx_id=0)

# Image Classification Model
classification_model = tf.keras.applications.ResNet50V2(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax'
)

# =====================
# Database Functions
# =====================
def init_db():
    """Initialize SQLite database"""
    conn = sqlite3.connect(DATABASE_FILE)
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS faces
                (id INTEGER PRIMARY KEY, name TEXT, embedding BLOB)''')
    conn.commit()
    conn.close()

def save_to_db(name, embedding):
    """Save face embedding to database"""
    conn = sqlite3.connect(DATABASE_FILE)
    c = conn.cursor()
    c.execute("INSERT INTO faces (name, embedding) VALUES (?, ?)",
             (name, embedding.tobytes()))
    conn.commit()
    conn.close()

def load_from_db():
    """Load all face embeddings from database"""
    conn = sqlite3.connect(DATABASE_FILE)
    c = conn.cursor()
    c.execute("SELECT name, embedding FROM faces")
    data = {row[0]: np.frombuffer(row[1], dtype=np.float32) for row in c.fetchall()}
    conn.close()
    return data

# =====================
# Image Processing
# =====================
def process_face(image):
    """Preprocess face image for embedding"""
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    return (image / 127.5) - 1.0

def get_embedding(image):
    """Generate face embedding"""
    faces = face_model.get(image)
    if faces:
        return faces[0].embedding
    return None

def classify_image(image):
    """Classify image using ResNet50V2"""
    img = cv2.resize(image, (224, 224))
    img = tf.keras.applications.resnet_v2.preprocess_input(img)
    preds = classification_model.predict(np.expand_dims(img, 0))
    return tf.keras.applications.resnet_v2.decode_predictions(preds, top=3)[0]

def similarity_search(query_img, threshold):
    """Search for similar faces in database"""
    embedding = get_embedding(query_img)
    if embedding is None:
        return []

    database = load_from_db()
    results = []
    for name, db_emb in database.items():
        similarity = cosine_similarity([embedding], [db_emb])[0][0]
        if similarity >= threshold:
            results.append((name, similarity))

    return sorted(results, key=lambda x: x[1], reverse=True)

def similarity_search_from_image(image_path, thresh):
    image = cv2.imread(image_path)
    blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0))
    face_detector.setInput(blob)
    detections = face_detector.forward()

    faces = []
    for i in range(detections.shape[2]):
        confidence = detections[0,0,i,2]
        if confidence > 0.9:
            box = detections[0,0,i,3:7] * np.array([image.shape[1], image.shape[0]]*2)
            faces.append(box.astype("int"))

    for (x, y, w, h) in faces:
        face_img = image[y:h, x:x+w]

    return similarity_search(face_img, thresh)

def webcam_face_identifier():
    init_db()
    os.makedirs(CAPTURE_DIR, exist_ok=True)

    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    db = load_from_db()
    last_capture = {}
    register_mode = False
    embeddings = []
    current_name = ""
    known_embs = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))
        face_detector.setInput(blob)
        detections = face_detector.forward()

        faces = []
        for i in range(detections.shape[2]):
            confidence = detections[0,0,i,2]
            if confidence > 0.9:
                box = detections[0,0,i,3:7] * np.array([frame.shape[1], frame.shape[0]]*2)
                faces.append(box.astype("int"))

        for (x, y, w, h) in faces:
            face_img = frame[y:h, x:x+w]

            if register_mode:
                embeddings.append(get_embedding(face_img))
                cv2.putText(frame, f"Captured: {len(embeddings)}/50", (x+5,y-30),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
            else:
                emb = get_embedding(face_img)
                if emb is None:
                    continue
                best_match = ("Unknown", 0.0)
                for name, db_emb in db.items():
                    similarity = cosine_similarity([emb], [db_emb])[0][0]
                    if similarity > best_match[1]:
                        best_match = (name, similarity)

                color = (0,255,0) if best_match[1] > FACE_THRESHOLD else (0,0,255)
                cv2.rectangle(frame, (x,y), (w,h), color, 2)
                cv2.putText(frame, f"{best_match[0]} ({best_match[1]:.0%})",
                           (x+5,y-30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        db = load_from_db()

        cv2.imshow('Smart Vision System', frame)

        key = cv2.waitKey(1)
        if key == ord('q'):
            break
        elif key == ord('s'):
            current_name = input("Enter name: ")
            register_mode = True
        elif key == ord(' ') and register_mode and len(embeddings) >= 50:
            avg_emb = np.mean(embeddings, axis=0)
            save_to_db(current_name, avg_emb)
            db = load_from_db()
            register_mode = False
            embeddings = []

    cap.release()
    cv2.destroyAllWindows()

def object_detection_from_webcam():
    model_path = 'yolo11m.pt'
    model = YOLO(model_path)

    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = model.predict(frame)

        for result in results:
            boxes = result.boxes
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0]
                class_id = box.cls[0]
                confidence = box.conf[0]

                if confidence > 0.5:
                    label = model.names[int(class_id)]
                    color = (0, 255, 0)
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
                    cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        cv2.imshow('Object Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

def object_detection_from_image(image_path):
    model_path = "yolo11m.pt"
    model = YOLO(model_path)

    img = cv2.imread(image_path)

    results = model.predict(img)

    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            class_id = box.cls[0]
            confidence = box.conf[0]

            if confidence > 0.5:
                label = model.names[int(class_id)]
                color = (0, 255, 0)
                cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
                cv2.putText(img, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    cv2.imshow('Object Detection', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/sina/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/sina/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/sina/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/sina/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/sina/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size

I0000 00:00:1741198083.850717  120767 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1741198083.851244  120767 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1741198083.851418  120767 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1741198083.947078  120767 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [3]:
webcam_face_identifier()

/home/sina/anaconda3/envs/tf/lib/python3.12/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4
/home/sina/anaconda3/envs/tf/lib/python3.12/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4
/home/sina/anaconda3/envs/tf/lib/python3.12/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter wi